In [7]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize  , wordpunct_tokenize , TweetTokenizer , regexp_tokenize 

from nltk.stem import PorterStemmer , SnowballStemmer , WordNetLemmatizer

from nltk.probability import FreqDist

from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer 

from wordcloud import WordCloud

from string import punctuation

from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [23]:
email = pd.read_csv( 'emails.txt' )
email.head(2)

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1


In [28]:
# email[email.duplicated()]
email.drop_duplicates( inplace = True)

In [42]:
def clean_txt(sent):
    sent = sent[9:]
    sent = sent.replace( "n ' t" , " not")
    tokens = word_tokenize(sent.lower())
    stem_s = SnowballStemmer('english') 
    stop_updated = list( set( stopwords.words("english") + list(punctuation)  ) - set( ['not' , 'nor'] ) )
    final_word = [stem_s.stem(term) for term in tokens if term not in stop_updated 
               and len(term) > 2] 
    res = " ".join(final_word)
    return res

In [44]:
email['text'] = email['text'].map( clean_txt )

In [72]:
email.head(2)

,text,spam
0,natur irresist corpor ident realli hard recoll...,1
1,stock trade gunsling fanni merril muzo not col...,1


In [93]:
from sklearn.model_selection import train_test_split 

x = email['text']
y = email['spam']

x_train , x_test , y_train , Y_test = train_test_split( x , y , test_size = 0.3 , random_state = 42 )

In [94]:
print( x_train.shape , x_test.shape )

(3986,) (1709,)


In [95]:
tfidf = TfidfVectorizer( max_features = 3500 )

x_train_tfidf = tfidf.fit_transform( x_train )
x_test_tfidf = tfidf.transform( x_test )


In [96]:
print( x_train_tfidf.shape , x_test_tfidf.shape )

(3986, 3500) (1709, 3500)


In [97]:
from sklearn.naive_bayes import MultinomialNB

In [101]:
classifier = MultinomialNB()
classifier.fit(x_train_tfidf , y_train )

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [103]:
pred = classifier.predict( x_test_tfidf )

In [104]:
from sklearn.metrics import accuracy_score , confusion_matrix

In [106]:
accuracy_score( Y_test , pred)

0.9859566998244588

In [107]:
confusion_matrix( Y_test , pred)

array([[1293,    5],
       [  19,  392]], dtype=int64)